In [1]:
def create_subwords(word, min_length=3):
    subwords = []
    length = len(word)
    
    for start in range(length):
        for end in range(start + min_length, length + 1):
            subword = word[start:end]
            subwords.append(subword)
    subwords[0] = "<" + subwords[0]
    subwords[-1]= subwords[-1]+ ">" 
    
    return subwords

word = "apple"
subwords = create_subwords(word)
print(subwords)

['<app', 'appl', 'apple', 'ppl', 'pple', 'ple>']


In [2]:
import torch.nn as nn

class HierarchicalSoftmax(nn.Module):
    def __init__(self, input_size, num_classes):
        super(HierarchicalSoftmax, self).__init__()
        self.num_classes = num_classes
        self.tree = self.build_tree(input_size)

    def build_tree(self, input_size):
        tree = nn.ModuleList()
        for i in range(self.num_classes):
            tree.append(nn.Sequential(
                nn.Linear(input_size, 1),  
                nn.Sigmoid(),
            ))
        return tree

    def forward(self, x):
        outputs = []
        for i in range(self.num_classes):
            output = self.tree[i](x)
            outputs.append(output)
        return torch.cat(outputs, dim=1)

In [3]:
import torch


input_size = 100
num_classes = 10
model = HierarchicalSoftmax(input_size, num_classes)

batch_size = 2
input_data = torch.randn(batch_size, input_size)

outputs = model(input_data)
print(outputs)

tensor([[0.3040, 0.2945, 0.3813, 0.4524, 0.5232, 0.5590, 0.4761, 0.5235, 0.3246,
         0.4809],
        [0.7309, 0.5524, 0.4305, 0.5879, 0.3448, 0.4747, 0.4499, 0.5043, 0.4370,
         0.5274]], grad_fn=<CatBackward0>)
